In [1]:
# Import
import re
import math
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.integrate import quad


In [2]:
# Parameters
n = 30
C = 4
meanT = 40
meanM = 15
oneItemST = 3

# The number of items in a customer's cart is an EXPONENTIAL random variable
def expCDF(x_, lambda_):
    return 1 - math.exp(- lambda_ * x_)

def expPDF(x_, lambda_):
    return lambda_ * math.exp(- lambda_ * x_)

# Factors
p = 0.5
K = 8

# CSV name
csv_name = "VTM_Exponential"

In [3]:
CSV = pd.read_csv(f"{csv_name}.csv").fillna(0)
CSV = CSV[CSV["type"] == "scalar"]

# NQ: Mean Number of Customers in Queue [Queues]
Exp_NQ = CSV[CSV["module"].str.contains("SuperMarket.checkout.queues")]
Exp_NQ = Exp_NQ[Exp_NQ["name"] == "queueLength:timeavg"]

# RHO: Mean Utilization [Queues]
Exp_RHO = CSV[CSV["module"].str.contains("SuperMarket.checkout.queues")]
Exp_RHO = Exp_RHO[Exp_RHO["name"] == "busy:timeavg"]

# S: Mean Service Time (t_S) [LocalSink] 
Exp_S = CSV[CSV["module"].str.contains("SuperMarket.checkout.localSink")]
Exp_S = Exp_S[Exp_S["name"] == "totalServiceTime:mean"]

# W: Mean Waiting Time [Queues]
Exp_W = CSV[CSV["module"].str.contains("SuperMarket.checkout.queues")]
Exp_W = Exp_W[Exp_W["name"] == "queueingTime:mean"]


# Dictionary for containing all the DataFrames
CSV_Dict = {
    "NQ": Exp_NQ,
    "RHO": Exp_RHO,
    "S": Exp_S,
    "W": Exp_W
}


PI = {}

Q = []
for i in range(C):
    Q.append(f"Q{i}")

for key in CSV_Dict:
    PI[key] = pd.DataFrame(columns=Q)
    for i in range(C):
        temp = []
        temp = [CSV_Dict[key]["value"].values[j*C + i] for j in range(n)]
        string = f"Q{i}"
        temp = list(map(float, temp))
        PI[key][string] = temp
   

In [4]:
PI_TABLE = {}

for key in PI:
    PI_TABLE[key] = pd.DataFrame(columns=["Theoretical", "Mean", "CI - LB", "CI - UB"])
    
    # Mean Value of the "n" repetitions for each Queue
    PI_TABLE[key]["Mean"] = PI[key].mean()

    # CI 95%
    # Lower Bound
    PI_TABLE[key]["CI - LB"] = PI[key].mean() - (1.96 * (PI[key].std() / math.sqrt(n)))
    # Upper Bound
    PI_TABLE[key]["CI - UB"] = PI[key].mean() + (1.96 * (PI[key].std() / math.sqrt(n)))



In [5]:
# for key in PI_TABLE:
#     print(f"[{key}]:")
#     print(PI_TABLE[key])
#     print("\n")

In [6]:
# Theoretical Formulas and Computations

# g(M) = round(M) + 1 = floor(M+1.5)
def g(x):
    return math.floor(x+1.5)  

def integrand(x):
    return (g(x) * expPDF(x, (1/(meanM-1))))

def integrand_2(x):
    return ((g(x)**2) * expPDF(x, (1/(meanM-1))))


# Number of Quick Tills
quickTills = int(p*C)

# Interarrival Rate
lambdaT = 1/meanT

temp = {}

if (quickTills == 0) or quickTills == C:
    # There are no Quick Tills or all the Tills are Quick Tills
    # So the probability alphaK of being routed to a Quick Till is either 0 or 1

    pi_ = 1/C
    lambdaT_ = lambdaT*pi_

    meanST_ = meanM * oneItemST
    lambdaST_ = 1/meanST_
    
    # Utilization
    rho_ = lambdaT_/lambdaST_

    # Mean Number of Customers in M/M/1
    N_ = rho_/(1-rho_)

    # Mean Number of Customers in a Queue
    NQ_ = N_ - rho_

    # Mean Response Time
    R_ = N_/lambdaT_

    # Mean Waiting Time
    W_ = R_ - meanST_


    temp["NQ"] = NQ_
    temp["RHO"] = rho_
    temp["S"] = meanST_
    temp["W"] = W_


    for key in PI_TABLE:
        PI_TABLE[key]["Theoretical"] = [temp[key]] * C

else:
    # There are Quick Tills AND Normal Tills
    # So the probability alphaK of being routed to a Quick Till is 0 < alphaK < 1
    
    # CDF of an EXPONENTIAL distribution
    alphaK = expCDF((K-0.5), (1/(meanM-1)))

    # --- Quick Tills ---

    # Probability of being routed to a specific Quick Till (i)
    # Inside a subsystem (Quick Tills or Normal Tills) the
    # probability of being routed to a specific Till is the
    # same (i.e. "equallylikely")
    pi_i = alphaK/quickTills
    
    # Interarrival Rate for an individual Quick Till
    lambdaT_i = lambdaT*pi_i
    
    # Mean Number of items in a customer's cart for an individual Quick Till
    meanM_i = (quad(integrand, 0, (K-0.5), limit=200)[0])/alphaK

    # Mean Service Time for an individual Quick Till
    meanST_i = meanM_i * oneItemST


    # Variance = E[X^2] - E[X]^2
    # Variance of M_i
    VarM_i = ((quad(integrand_2, 0, (K-0.5), limit=200)[0])/alphaK) - meanM_i**2

    # Variance of ST_i
    VarST_i = VarM_i * oneItemST**2
    
    
    # Performance Indeces for an individual Quick Till

    rho_i = lambdaT_i * meanST_i
    N_i = rho_i + ((rho_i**2 + lambdaT_i**2 * VarST_i)/(2*(1-rho_i)))
    NQ_i = N_i - rho_i
    R_i = N_i/lambdaT_i
    W_i = R_i - meanST_i
    
    temp["NQ"] = {}
    temp["NQ"]["i"] = NQ_i

    temp["RHO"] = {}
    temp["RHO"]["i"] = rho_i

    temp["S"] = {}
    temp["S"]["i"] = meanST_i

    temp["W"] = {}
    temp["W"]["i"] = W_i


    # --- Normal Tills ---

    # Probability of being routed to a specific Normal Till (j)
    # Inside a subsystem (Quick Tills or Normal Tills) the
    # probability of being routed to a specific Till is the
    # same (i.e. "equallylikely")
    pi_j = (1-alphaK)/(C-quickTills)

    # Interarrival Rate for an individual Normal Till
    lambdaT_j = lambdaT*pi_j

    # Mean Number of items in a customer's cart for an individual Normal Till
    meanM_j = (quad(integrand, (K-0.5), np.inf, limit=200)[0])/(1-alphaK)

    # Mean Service Time for an individual Normal Till
    meanST_j = meanM_j * oneItemST

    # Variance = E[X^2] - E[X]^2
    # Variance of M_j
    VarM_j = ((quad(integrand_2, (K-0.5), np.inf, limit=200)[0])/(1-alphaK)) - meanM_j**2

    # Variance of ST_j
    VarST_j = VarM_j * oneItemST**2
    
    # Performance Indeces for an individual Normal Till

    rho_j = lambdaT_j * meanST_j
    N_j = rho_j + ((rho_j**2 + lambdaT_j**2 * VarST_j)/(2*(1-rho_j)))
    NQ_j = N_j - rho_j
    R_j = N_j/lambdaT_j
    W_j = R_j - meanST_j

    temp["NQ"]["j"] = NQ_j
    temp["RHO"]["j"] = rho_j
    temp["S"]["j"] = meanST_j
    temp["W"]["j"] = W_j
    
    # Adding the Theoretical Values to the PI_TABLE
    for key in PI_TABLE:
        PI_TABLE[key]["Theoretical"] = [temp[key]["i"]] * quickTills + [temp[key]["j"]] * (C-quickTills)

    


C:\Users\giova\AppData\Local\Temp\ipykernel_22060\33160946.py:123: IntegrationWarning: The maximum number of subdivisions (200) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.
  meanM_j = (quad(integrand, (K-0.5), np.inf, limit=200)[0])/(1-alphaK)
C:\Users\giova\AppData\Local\Temp\ipykernel_22060\33160946.py:130: IntegrationWarning: The maximum number of subdivisions (200) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitt

In [7]:
for key in PI_TABLE:
    PI_TABLE[key]["OK"] = [""] * C
    for i in range(C):
        if PI_TABLE[key]["Theoretical"].iloc[i] <= PI_TABLE[key]["CI - UB"].iloc[i] and PI_TABLE[key]["Theoretical"].iloc[i] >= PI_TABLE[key]["CI - LB"].iloc[i]:
            PI_TABLE[key].loc[f"Q{i}", "OK"] = "V"

In [8]:
for key in PI_TABLE:
    print(f"[{key}]:")
    print(PI_TABLE[key])
    print("\n")

[NQ]:
    Theoretical      Mean   CI - LB   CI - UB OK
Q0     0.003123  0.002995  0.002761  0.003230  V
Q1     0.003123  0.003402  0.003119  0.003684  V
Q2     0.334318  0.325612  0.304830  0.346395  V
Q3     0.334318  0.346722  0.322885  0.370560  V


[RHO]:
    Theoretical      Mean   CI - LB   CI - UB OK
Q0     0.068452  0.068394  0.067328  0.069459  V
Q1     0.068452  0.068459  0.066834  0.070084  V
Q2     0.493988  0.492145  0.485497  0.498793  V
Q3     0.493988  0.496031  0.486992  0.505071  V


[S]:
    Theoretical       Mean    CI - LB    CI - UB OK
Q0    13.203587  13.226692  13.111825  13.341558  V
Q1    13.203587  13.106028  13.003293  13.208762  V
Q2    67.524951  67.018288  66.366837  67.669739  V
Q3    67.524951  67.810909  67.239446  68.382372  V


[W]:
    Theoretical       Mean    CI - LB    CI - UB OK
Q0     0.602429   0.578014   0.534902   0.621127  V
Q1     0.602429   0.648275   0.601448   0.695101  V
Q2    45.699115  44.209815  41.519669  46.899961  V
Q3    45.6991

In [9]:
# alphaK is in the denominator because
# it is there to normalize the PDF. Indeed
# calculating the integral of the truncated
# PDF from 0 to K-0.5 gives exactly 1

def integrand_Test(x):
    return expPDF(x, (1/(meanM-1)))

meanM_i = (quad(integrand_Test, 0, (K-0.5), limit=200)[0])/alphaK
meanM_i

1.0000000000000002

In [10]:
# How the mean of M chages when the different transformation are applied

meanM_Test = 20


# meanM_Test -1
def integrand_Test1(x):
    return (x*expPDF(x, (1/(meanM_Test-1))))

# meanM_Test
def integrand_Test2(x):
    return (x*expPDF(x, (1/(meanM_Test))))

# mean of M'
def integrand_Test3(x):
    return (math.floor(x+0.5)*expPDF(x, (1/(meanM_Test))))

# mean of M''
def integrand_Test4(x):
    return (math.floor(x+1+0.5)*expPDF(x, (1/(meanM_Test))))

# mean of M'' with meanM_Test - 1
def integrand_Test5(x):
    return (math.floor(x+1+0.5)*expPDF(x, (1/(meanM_Test-1))))



print("meanM_Test - 1: " + str(quad(integrand_Test1, 0, np.inf, limit=200)[0]))
print("meanM_Test: " + str(quad(integrand_Test2, 0, np.inf, limit=200)[0]))
print("mean of M': " + str(quad(integrand_Test3, 0, np.inf, limit=200)[0]))
print("mean of M'': " + str(quad(integrand_Test4, 0, np.inf, limit=200)[0]))
print("mean of M'' with meanM_Test - 1: " + str(quad(integrand_Test5, 0, np.inf, limit=200)[0]))


meanM_Test - 1: 19.00000000000003
meanM_Test: 19.999999999999787
mean of M': 19.997687352603702
mean of M'': 20.99768735260366
mean of M'' with meanM_Test - 1: 19.997728271654925


C:\Users\giova\AppData\Local\Temp\ipykernel_22060\2955862547.py:30: IntegrationWarning: The maximum number of subdivisions (200) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.
  print("mean of M': " + str(quad(integrand_Test3, 0, np.inf, limit=200)[0]))
C:\Users\giova\AppData\Local\Temp\ipykernel_22060\2955862547.py:31: IntegrationWarning: The maximum number of subdivisions (200) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain fro